## Introduction

So far in this course, we've considered the general *supervised learning* scenario, in which we are given a feature matrix $\mX \in \R^{n\times p}$ and a target vector $\vy \in \R^n$. We then solve the empirical risk minimization problem in order to choose model parameters that minimize a loss function on the training data. The exact structure of this loss function depends on things like whether we are doing classification or regression, what our computational resources are, and other considerations. 

But feature matrices $\mX$ and target vectors $\vy$ don't just exist in the world: they are *collected* and *measured*. We can think of data collection and measurement as posing three fundamental questions: 

- **Data collection**: Which **rows** (observations) exist in $\mX$ and $\vy$? 
- **Measurement**: which **columns** (features) exist in $\mX$? 
- **Measurement**: what is the **target** $\vy$ and how is it measured? 

Broadly, we can think of the complete machine learning workflow as having phases corresponding to problem definition, data collection + measurement, modeling, and evaluation. Here's roughly how this looks: 

```{mermaid}
flowchart TB

    subgraph problem[problem definition]
        need[identify need]-->design_collection[design data collection]
    end
    subgraph measurement[data collection + measurement]
        training[training data] 
        testing[testing data]
    end
    subgraph modeling
        explore[explore data] --> engineer[engineer features]
        engineer --> design[design model]
    end
    subgraph assessment
        test --> audit
        audit --> deploy
        deploy-->evaluate
    end
    design_collection-->measurement
    training --vectorization--> modeling
    design --> assessment
    testing --vectorization--> assessment
    need-->assessment

```

So far, we've spent most of our time in the "modeling" module, especially the last two steps. We've also studied some of the ways to test and audit algorithms. Today we're going to discuss **vectorization**. We can think of vectorization as what happens *between* the collection of raw data and the use of that data as input for models.  

::: {.callout-note}

::: {#def-vectorization}

## Vectorization

**Vectorization** is the act of assigning to each data observation a vector $\vx$, thus forming a feature matrix $\mX$. Formally, a **vectorization map** is a function $v:\cD\rightarrow \R^p$ such that, if $d \in \cD$ is a data observation, then $\vx = v(d)$ is a set of features corresponding to $d$. 

:::
:::

The reason that vectorization is necessary is that **machine learning models only understand numbers**. So, if our data *isn't* numbers, we need to convert it into numbers in order to use it for modeling. 

## What Data Needs Vectorization?

Most of it! 

- If your data comes to you as a table or matrix containing only numbers, in which each row corresponds to exactly one observation, then you may not need to vectorize. 
- If your data comes to you in *any other form*, then you need to vectorize. 

Some data that usually require vectorization:

- Images
- Text
- Audio files
- Most genomic data
- Etc. etc. 

There are tons of ways of vectorizing different kinds of data, and we're not going to cover all of them. Instead, we're going to go a little more in depth on **text vectorization**. We'll discuss image vectorization much more when we get to convolutional neural networks. [For your projects, depending on the data you want to work with, you may need to research vectorization schemes appropriate to your data.]{.aside}

# Case Study: Sentiment Analysis of COVID-19 Tweets

Instead of discussing text vectorization in the abstract, let's jump straight into an example. *Sentiment analysis* describes modeling techniques that aim to describe the emotional valence of text. For example, sentiment analysis is often used to automatically describe text as "positive"/"happy" or "negative"/"sad". The function below will download and return a set of training data used for sentiment analysis of tweets related to the COVID-19 pandemic. [I retrieved this data from its [original posting](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification) on Kaggle.]{.aside} 

In [ ]:
import pandas as pd

def grab_tweets(data_set = "train"):
    url = f"https://raw.githubusercontent.com/PhilChodrow/PIC16A/master/datasets/Corona_NLP_{data_set}.csv"
    df = pd.read_csv(url, encoding='iso-8859-1') 
    df = df[["OriginalTweet", "Sentiment"]]
    return df
    
df_train = grab_tweets()

Let's take a look at our training data: 

::: {.callout-warning}

## Activity

Chat with your group. What are **three questions** you have about how the data was collected? 

:::

## Sketchy Labels

These tweets were [labeled manually](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification/discussion/186907) by the original collector of the data. As with any setting in which humans need to make subjective decisions, there is considerable possibility for debate. For example, here is one tweet that was labeld "**extremely positive**":

Challenges that can cause sketchy labels include:

- Speed of labeling (it takes a LONG time to make high-quality labels)
- Language familiarity
- Ambiguity in the target language
- Lots more!

Almost always, when working with real-world data sets, we need to keep in mind that not only is our model approximate and our data incomplete, but the data may also be contaminated with *errors* that we aren't really able to control. [See @northcutt2021labelerrors for much more on label errors in common machine learning benchmarks.]{.aside}

## Target Vectorization

Our aim is to predict the `Sentiment` in terms of the text of the `OriginalTweet`. However, neither the text `OriginalTweet` nor the target `Sentiment` are numbers. So, we need to vectorize. 

The possible values of the `Sentiment` column are 

Vectorizing the target `Sentiment` is simple (although there are multiple ways). We'll construct a new target vector which is `1` if the sentiment is `Positive` or `Extremely Positive` and `0` otherwise: 

Vectorizing the predictor `OriginalTweet` is much more complicated, and here we face a number of choices. 

### Term Frequency (TF) Vectorization

In natural language processing (NLP), a data set of text is often called a *corpus*, and each observation is often called a *document*. Here, each document is a tweet. 

One standard vectorization technique is to construct a *term-document matrix*. In a term-document matrix, each row corresponds to a document and each column corresponds to a "term" (usually a word) that is present in the document. The entry $x_{ij}$ of this matrix is the number of terms that term $j$ appears in document $i$, which we'll call $\mathrm{tf}_{ij}$. To construct a term-document matrix, we can use the `CountVectorizer` from `sklearn`. 

Here, `max_df` and `min_df` specify a range of frequencies to include. If a term is present in almost all documents (like "the" or "of"), then this term may not be a good indication of sentiment. On the other hand, if a term appears in only one or two documents, we probably don't have enough data to figure out whether it matters. Finally, the choice of `stop_words` tells our vectorizer to ignore common English words that are unlikely to carry much emotional meaning, like "and" or "if". 

Here's our term-document matrix. Note that most of the entries are 0 because tweets are so short! 

The function below summarizes our entire data prep pipeline, which we'll need for when we get to the test set. 

In [ ]:
def prep_tweets(df, vectorizer, train = True):
    if train: 
        vectorizer.fit(df_train["OriginalTweet"])
    X = vectorizer.transform(df["OriginalTweet"]) # term-document matrix
    y = 1*df["Sentiment"].str.contains("Positive")

    return X, y

### First Model

Let's check on the base rate: 

So, always guessing that a tweet is *not* positive would be correct 56% of the time. Let's see if we can beat this using logistic regression. 

This model achieves 87% accuracy on the training data. 

### Inverse Document Frequency Weighting 

Simple term-document matrices are good for some tasks, but in other cases it is useful to downweight terms according to their frequency in the overall training corpus. This allows our models to place greater emphasis on rarer terms, which might be more expressive of strong emotions. 

In term-frequency-inverse-document-frequency (TF-IDF) weighting, the entry for term $j$ in document $i$ is [Exact details of TF-IDF weightings differ; this is the one implemented by default in `sklearn`.]{.aside} 

$$
\tilde{\mathrm{x}}_{ij} = \overbrace{\mathrm{tf}_{ij}}^{\text{Term frequency}}\times \underbrace{\mathrm{idf}_i}_{\text{inverse document frequency}}\;. 
$$

Here, the *term frequency* $\mathrm{tf}_{ij}$ is again the number of times that term $i$ appears in document $j$, while the inverse document frequency $\mathrm{idf}_i$ is computed with the formula 

$$
\mathrm{idf}_i = \log \frac{1+n}{1+\mathrm{df}_i} + 1\;
$$
with $\mathrm{df}_i$ being the total number of documents in which term $i$ appears. Finally, each row of $\tilde{\mathrm{x}}_{ij}$ is normalized to have unit length: 

$$
x_{ij} = \frac{x_{ij}}{\sqrt{\sum_{j}x_{ij}^2}}
$$

These $x_{ij}$ are then collected to form the feature matrix $\mX$. Let's try constructing a model using TF-IDF vectorization: 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(max_df = 0.2, min_df = 0.001, stop_words = 'english')
X_train_tfidf, y_train = prep_tweets(df_train, tfidfv, train = True)

In [ ]:
LR_tfidf = LogisticRegression()
LR_tfidf.fit(X_train_tfidf, y_train)
LR_tfidf.score(X_train_tfidf, y_train)

Our TF-IDF model got a lower training score. At this stage, one good approach would be to choose which vectorization to use (as well as the vectorization parameters) using cross-validation. For now, we'll just go ahead and grab the test set: 

In [ ]:
df_test = grab_tweets(data_set = "test")
X_test_cv, y_test = prep_tweets(df_test, vectorizer = cv, train = False)
X_test_tfidf, y_test = prep_tweets(df_test, vectorizer = tfidfv, train = False)

And evaluate!

In [ ]:
print("Term-Document Frequency")
print(LR_cv.score(X_test_cv, y_test))
print("TF-IDF")
print(LR_tfidf.score(X_test_tfidf, y_test))

In this case, TF-IDF did a little worse than term-document frequency vectorization on the test set. 

## Model Inspection

Let's take a moment to learn more about how our term-document frequency-based model looks at the data. One good way to do this is by looking at the confusion matrices: 

The false negative rate is higher than the true positive rate, suggesting that our model tends to tilt negative. Let's take a look at some tweets that our model labeled as negative even though the label was positive: 

At this point we might have some further questions for the producer of this data set about how he did the labeling: don't some of these tweets look like they "really" should be negative? 


## Word-Based Sentiment Analysis

A nice feature of linear models like logistic regression is that we can actually check the coefficient for each word in the model. This coefficient can give us important information about which words the model believes are most positive  or most negative. One easy way to get at this information is to construct a data frame with the coefficients and the words: 

Now we can obtain positive and negative words by sorting. Here are some of the good ones: 

On the other hand, here are some of the negative ones: 

A common use for these coefficients is to assign sentiment scores to sentences. Here's a function that does this. It works by first stripping the punctuation and capitalization from a string, and then looking up each of its individual words in a dictionary.  

In [ ]:
from string import punctuation 

d = {coef_df["word"].loc[i] : coef_df["coef"].loc[i] for i in coef_df.index}

def sentiment_of_string(s):
    no_punc = s
    for punc in punctuation:
        no_punc = no_punc.replace(punc, "")
    
    words = no_punc.lower().split()
    return np.mean([d[word] for word in words if word in d ])

This approach is the basis of [The Hedonometer](https://hedonometer.org/timeseries/en_all/?from=2021-09-23&to=2023-03-22), a large-scale Twitter sentiment analysis tool from our friends at the University of Vermont. 






::: {callout-warning}

## Activity

There is a very important kind of information that is *not* captured by term-document matrices, even with inverse-document-frequency weighting. Consider the following two sentences: 

1. "I like pears, not apples."
2. "I like apples, not pears."

Would these sentences have different representations in a term-document matrix?

:::
